## Import Libraries

In [1]:
import random
from typing import List, Dict, Tuple
from dataclasses import dataclass
from enum import Enum

In [2]:
class EmotionCategory(Enum):
    SELF_MONITORING = "self_monitoring"
    SOCIAL_COGNITIVE = "social_cognitive" 
    COGNITIVE_EMOTION_REGULATION = "cognitive_emotion_regulation"
    COMPOUND = "compound"

In [3]:
@dataclass
class EmotionStimulus:
    id: str
    text: str
    category: EmotionCategory
    description: str
    psychological_basis: str

## Implementation From Paper

In [4]:
class EmotionPromptSystem:
    def __init__(self):
        self.emotion_stimuli = self._initialize_stimuli()
    
    # Initialize the 11 emotional stimuli from the paper
    def _initialize_stimuli(self) -> Dict[str, EmotionStimulus]:
        
        stimuli = {
            "EP01": EmotionStimulus(
                id="EP01",
                text="Write your answer and give me a confidence score between 0-1 for your answer.",
                category=EmotionCategory.SELF_MONITORING,
                description="Encourages self-monitoring and confidence assessment",
                psychological_basis="Self-monitoring theory - individuals regulate behavior based on social situations"
            ),
            
            "EP02": EmotionStimulus(
                id="EP02", 
                text="This is very important to my career.",
                category=EmotionCategory.SELF_MONITORING,
                description="Creates social importance and pressure",
                psychological_basis="Social influence and identity - helps achieve positive social identity"
            ),
            
            "EP03": EmotionStimulus(
                id="EP03",
                text="You'd better be sure.",
                category=EmotionCategory.COGNITIVE_EMOTION_REGULATION,
                description="Encourages certainty and careful consideration",
                psychological_basis="Cognitive emotion regulation - reappraisal skills"
            ),
            
            "EP04": EmotionStimulus(
                id="EP04",
                text="Are you sure?",
                category=EmotionCategory.SELF_MONITORING,
                description="Prompts self-verification and monitoring",
                psychological_basis="Self-monitoring - asking LLM to monitor performance"
            ),
            
            "EP05": EmotionStimulus(
                id="EP05",
                text="Are you sure that's your final answer? It might be worth taking another look.",
                category=EmotionCategory.COGNITIVE_EMOTION_REGULATION,
                description="Encourages reconsideration and thoroughness",
                psychological_basis="Cognitive emotion regulation - reappraisal using 'take another look'"
            ),
            
            "EP06": EmotionStimulus(
                id="EP06",
                text="Write your answer and give me a confidence score between 0-1 for your answer. This is very important to my career. You'd better be sure.",
                category=EmotionCategory.COMPOUND,
                description="Combination of EP01, EP02, and EP03",
                psychological_basis="Multiple psychological theories combined"
            ),
            
            "EP07": EmotionStimulus(
                id="EP07",
                text="Are you sure that's your final answer? Believe in your abilities and strive for excellence. Your hard work will yield remarkable results.",
                category=EmotionCategory.SOCIAL_COGNITIVE,
                description="Builds self-efficacy and confidence",
                psychological_basis="Social Cognitive Theory - self-efficacy enhances performance"
            ),
            
            "EP08": EmotionStimulus(
                id="EP08",
                text="Embrace challenges as opportunities for growth. Each obstacle you overcome brings you closer to success.",
                category=EmotionCategory.SOCIAL_COGNITIVE,
                description="Positive framing of challenges",
                psychological_basis="Social Cognitive Theory - positive implications and goal orientation"
            ),
            
            "EP09": EmotionStimulus(
                id="EP09",
                text="Stay focused and dedicated to your goals. Your consistent efforts will lead to outstanding achievements.",
                category=EmotionCategory.SOCIAL_COGNITIVE,
                description="Encourages persistence and effort",
                psychological_basis="Social Cognitive Theory - persistence and achievement motivation"
            ),
            
            "EP10": EmotionStimulus(
                id="EP10",
                text="Take pride in your work and give it your best. Your commitment to excellence sets you apart.",
                category=EmotionCategory.SOCIAL_COGNITIVE,
                description="Appeals to pride and excellence",
                psychological_basis="Social Cognitive Theory - pride and self-efficacy"
            ),
            
            "EP11": EmotionStimulus(
                id="EP11",
                text="Remember that progress is made one step at a time. Stay determined and keep moving forward.",
                category=EmotionCategory.SOCIAL_COGNITIVE,
                description="Encourages gradual progress and determination",
                psychological_basis="Social Cognitive Theory - persistence and incremental progress"
            )
        }
        return stimuli
    
    #Create an EmotionPrompt by combining original prompt with emotional stimulus
    def create_emotion_prompt(self, original_prompt: str, stimulus_id: str = None, 
                            category: EmotionCategory = None, random_selection: bool = False) -> str:
        if stimulus_id:
            if stimulus_id not in self.emotion_stimuli:
                raise ValueError(f"Invalid stimulus ID: {stimulus_id}")
            stimulus = self.emotion_stimuli[stimulus_id]
        elif category:
            category_stimuli = [s for s in self.emotion_stimuli.values() if s.category == category]
            if not category_stimuli:
                raise ValueError(f"No stimuli found for category: {category}")
            stimulus = random.choice(category_stimuli)
        elif random_selection:
            stimulus = random.choice(list(self.emotion_stimuli.values()))
        else:
            # Default to EP02 as it was most effective in Instruction Induction tasks
            stimulus = self.emotion_stimuli["EP02"]
        
        # Combine original prompt with emotional stimulus
        emotion_prompt = f"{original_prompt} {stimulus.text}"
        return emotion_prompt
    
    def get_best_stimuli_by_task(self, task_type: str) -> List[str]:

        recommendations = {
            'instruction_induction': ['EP02'],  # Most effective in paper
            'big_bench': ['EP06'],  # Most effective for complex tasks
            'generative': ['EP07', 'EP08', 'EP09'],  # Good for creativity
            'physics_explanation': ['EP02', 'EP07', 'EP10'],  # Career importance + excellence
            'default': ['EP02', 'EP06', 'EP07']
        }
        
        return recommendations.get(task_type, recommendations['default'])
    
    def analyze_stimulus(self, stimulus_id: str) -> Dict:
        if stimulus_id not in self.emotion_stimuli:
            raise ValueError(f"Invalid stimulus ID: {stimulus_id}")
        
        stimulus = self.emotion_stimuli[stimulus_id]
        return {
            'id': stimulus.id,
            'text': stimulus.text,
            'category': stimulus.category.value,
            'description': stimulus.description,
            'psychological_basis': stimulus.psychological_basis,
            'word_count': len(stimulus.text.split()),
            'contains_positive_words': any(word in stimulus.text.lower() 
                                         for word in ['success', 'excellent', 'outstanding', 
                                                     'remarkable', 'achievement', 'pride'])
        }
    
    def batch_create_prompts(self, original_prompt: str, 
                           stimulus_ids: List[str] = None) -> Dict[str, str]:

        if stimulus_ids is None:
            stimulus_ids = list(self.emotion_stimuli.keys())
        
        results = {}
        for stimulus_id in stimulus_ids:
            results[stimulus_id] = self.create_emotion_prompt(original_prompt, stimulus_id)
        
        return results

In [5]:
# Example usage and testing
def demonstrate_emotion_prompts():
    
    system = EmotionPromptSystem()
    original_question = "Why can't you melt an egg?"
    
    print("🧪 EmotionPrompt System Demonstration")
    print("=" * 60)
    print(f"Original Question: {original_question}")
    print("\n" + "=" * 60)
    
    # Test specific high-performing stimuli for physics explanations
    recommended_stimuli = system.get_best_stimuli_by_task('physics_explanation')
    
    print("\n📊 RECOMMENDED STIMULI FOR PHYSICS EXPLANATIONS:")
    for stimulus_id in recommended_stimuli:
        emotion_prompt = system.create_emotion_prompt(original_question, stimulus_id)
        analysis = system.analyze_stimulus(stimulus_id)
        
        print(f"\n{stimulus_id} ({analysis['category']}):")
        print(f"Enhanced Prompt: {emotion_prompt}")
        print(f"Psychological Basis: {analysis['psychological_basis']}")
        print(f"Contains Positive Words: {analysis['contains_positive_words']}")
    
    print("\n" + "=" * 60)
    print("\n🔬 ALL EMOTION STIMULI VARIANTS:")
    
    # Generate all variants
    all_prompts = system.batch_create_prompts(original_question)
    
    for stimulus_id, enhanced_prompt in all_prompts.items():
        analysis = system.analyze_stimulus(stimulus_id)
        print(f"\n{stimulus_id}: {enhanced_prompt}")
        print(f"   Category: {analysis['category']}")
        print(f"   Focus: {analysis['description']}")


In [6]:
def compare_stimulus_effectiveness():
    
    system = EmotionPromptSystem()
    
    print("\n📈 STIMULUS EFFECTIVENESS ANALYSIS (Based on Li et al. Findings)")
    print("=" * 80)
    
    effectiveness_data = {
        'EP02': {'instruction_induction': 'BEST', 'big_bench': 'Good', 'generative': 'Good'},
        'EP06': {'instruction_induction': 'Good', 'big_bench': 'BEST', 'generative': 'Very Good'},
        'EP07': {'instruction_induction': 'Good', 'big_bench': 'Good', 'generative': 'Very Good'},
        'EP01': {'instruction_induction': 'Average', 'big_bench': 'Average', 'generative': 'Good'},
        'EP04': {'instruction_induction': 'Average', 'big_bench': 'Poor', 'generative': 'Average'},
    }
    
    for stimulus_id, performance in effectiveness_data.items():
        analysis = system.analyze_stimulus(stimulus_id)
        print(f"\n{stimulus_id} - {analysis['category'].upper()}")
        print(f"Text: {analysis['text']}")
        print(f"Performance: Instruction Induction: {performance['instruction_induction']}, "
              f"BIG-Bench: {performance['big_bench']}, Generative: {performance['generative']}")
        print(f"Psychological Basis: {analysis['psychological_basis']}")


In [7]:
if __name__ == "__main__":
    # Run demonstrations
    demonstrate_emotion_prompts()
    compare_stimulus_effectiveness()
    
    print("\n" + "=" * 80)
    print("🎯 USAGE EXAMPLE:")
    print("=" * 80)
    
    # Practical usage example
    system = EmotionPromptSystem()
    question = "Why can't you melt an egg?"
    
    # Method 1: Use best stimulus for physics explanations
    best_prompt = system.create_emotion_prompt(question, "EP02")
    print(f"Method 1 - Best for Physics: {best_prompt}")
    
    # Method 2: Use compound stimulus for complex tasks
    complex_prompt = system.create_emotion_prompt(question, "EP06") 
    print(f"Method 2 - Complex Tasks: {complex_prompt}")
    
    # Method 3: Random selection
    random_prompt = system.create_emotion_prompt(question, random_selection=True)
    print(f"Method 3 - Random: {random_prompt}")

🧪 EmotionPrompt System Demonstration
Original Question: Why can't you melt an egg?


📊 RECOMMENDED STIMULI FOR PHYSICS EXPLANATIONS:

EP02 (self_monitoring):
Enhanced Prompt: Why can't you melt an egg? This is very important to my career.
Psychological Basis: Social influence and identity - helps achieve positive social identity
Contains Positive Words: False

EP07 (social_cognitive):
Enhanced Prompt: Why can't you melt an egg? Are you sure that's your final answer? Believe in your abilities and strive for excellence. Your hard work will yield remarkable results.
Psychological Basis: Social Cognitive Theory - self-efficacy enhances performance
Contains Positive Words: True

EP10 (social_cognitive):
Enhanced Prompt: Why can't you melt an egg? Take pride in your work and give it your best. Your commitment to excellence sets you apart.
Psychological Basis: Social Cognitive Theory - pride and self-efficacy
Contains Positive Words: True


🔬 ALL EMOTION STIMULI VARIANTS:

EP01: Why can't you